In [1]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
DATABASE_URL = os.getenv("DATABASE_URL")

In [3]:
# Menu data
menu_data = {
    "Item ID": [
        201, 202, 203, 204, 301, 302, 303, 
        401, 402, 403, 501, 502, 601, 602, 603
    ],
    "Category": [
        "Classic Pizzas", "Classic Pizzas", "Classic Pizzas", "Classic Pizzas", 
        "Specialty Pizzas", "Specialty Pizzas", "Specialty Pizzas", 
        "Appetizers", "Appetizers", "Appetizers", 
        "Desserts", "Desserts", 
        "Beverages", "Beverages", "Beverages"
    ],
    "Item Name": [
        "Margherita", "Pepperoni", "Veggie Supreme", "Hawaiian", 
        "BBQ Chicken", "Meat Feast", "Four Cheese", 
        "Garlic Breadsticks", "Mozzarella Sticks", "Chicken Wings", 
        "Chocolate Lava Cake", "Cinnamon Rolls", 
        "Coca-Cola", "Lemon Iced Tea", "Bottled Water"
    ],
    "Description": [
        "Classic pizza with mozzarella and fresh basil",
        "Loaded with pepperoni slices and melted cheese",
        "Topped with bell peppers, onions, olives, and mushrooms",
        "Ham and pineapple on a cheesy base",
        "Grilled chicken with tangy BBQ sauce",
        "Pepperoni, sausage, ham, and beef",
        "A cheesy delight with mozzarella, cheddar, parmesan, and gouda",
        "Warm breadsticks with garlic butter",
        "Fried mozzarella sticks with marinara sauce",
        "Choice of BBQ, Buffalo, or Plain",
        "Warm cake with gooey chocolate center",
        "Sweet rolls with cinnamon and icing",
        "Classic soda",
        "Refreshing iced tea with lemon flavor",
        "Pure bottled water"
    ],
    "Size Options": [
        ["Small", "Medium", "Large"], ["Small", "Medium", "Large"], ["Small", "Medium", "Large"], ["Small", "Medium", "Large"],
        ["Small", "Medium", "Large"], ["Small", "Medium", "Large"], ["Small", "Medium", "Large"],
        None, None, None,
        None, None,
        None, None, None
    ],
    "Price (Small)": [
        8.00, 10.00, 9.00, 10.00, 12.00, 13.00, 11.00, 
        5.00, 6.00, 7.50, 6.00, 5.50, 2.50, 3.00, 1.50
    ],
    "Price (Medium)": [
        10.00, 12.00, 11.00, 12.00, 14.00, 15.00, 13.00, 
        None, None, None, None, None, None, None, None
    ],
    "Price (Large)": [
        14.00, 16.00, 15.00, 16.00, 18.00, 19.00, 17.00, 
        None, None, None, None, None, None, None, None
    ]
}

# Create DataFrame
menu_df = pd.DataFrame(menu_data)

# Display the DataFrame
menu_df

,Item ID,Category,Item Name,Description,Size Options,Price (Small),Price (Medium),Price (Large)
0,201,Classic Pizzas,Margherita,Classic pizza with mozzarella and fresh basil,"[Small, Medium, Large]",8.0,10.0,14.0
1,202,Classic Pizzas,Pepperoni,Loaded with pepperoni slices and melted cheese,"[Small, Medium, Large]",10.0,12.0,16.0
2,203,Classic Pizzas,Veggie Supreme,"Topped with bell peppers, onions, olives, and ...","[Small, Medium, Large]",9.0,11.0,15.0
3,204,Classic Pizzas,Hawaiian,Ham and pineapple on a cheesy base,"[Small, Medium, Large]",10.0,12.0,16.0
4,301,Specialty Pizzas,BBQ Chicken,Grilled chicken with tangy BBQ sauce,"[Small, Medium, Large]",12.0,14.0,18.0
5,302,Specialty Pizzas,Meat Feast,"Pepperoni, sausage, ham, and beef","[Small, Medium, Large]",13.0,15.0,19.0
6,303,Specialty Pizzas,Four Cheese,"A cheesy delight with mozzarella, cheddar, par...","[Small, Medium, Large]",11.0,13.0,17.0
7,401,Appetizers,Garlic Breadsticks,Warm breadsticks with garlic butter,None,5.0,NaN,NaN
8,402,Appetizers,Mozzarella Sticks,Fried mozzarella sticks with marinara sauce,None,6.0,NaN,NaN
9,403,Appetizers,Chicken Wings,"Choice of BBQ, Buffalo, or Plain",None,7.5,NaN,NaN


In [4]:
# # Menu data
menu_data = list(menu_df.itertuples(index=False, name=None))

# Establish a connection to the database
try:
    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    # Drop existed table
    cursor.execute("DROP TABLE IF EXISTS order_items CASCADE;")
    cursor.execute("DROP TABLE IF EXISTS orders CASCADE;")
    cursor.execute("DROP TABLE IF EXISTS menu CASCADE;")
    print("Drop all table successfully")
    # Create the menu table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS menu (
            item_id INT PRIMARY KEY,
            category VARCHAR(50),
            item_name VARCHAR(100),
            description TEXT,
            size_options TEXT,
            price_small NUMERIC(5, 2),
            price_medium NUMERIC(5, 2),
            price_large NUMERIC(5, 2)
        );
    """)
    print("Table menu created successfully.")

    # Set the session's time zone to UTC+7
    cursor.execute("SET TIME ZONE 'UTC+7';")

    # Create the orders table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
            order_id SERIAL PRIMARY KEY,
            customer_name VARCHAR(100) NOT NULL,
            order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("Table orders created successfully.")

    # Create the order_items  table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS order_items  (
            order_item_id SERIAL PRIMARY KEY,
            order_id INT REFERENCES orders(order_id) ON DELETE CASCADE,
            item_id INT REFERENCES menu(item_id) ON DELETE CASCADE,
            quantity INT NOT NULL CHECK (quantity > 0),
            total_price NUMERIC(10, 2) NOT NULL CHECK (total_price >= 0)
        );
    """)
    print("Table order_items created successfully.")

    # Insert data into the table
    insert_query = """
        INSERT INTO menu (item_id, category, item_name, description, size_options, price_small, price_medium, price_large)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (item_id) DO NOTHING;
    """
    cursor.executemany(insert_query, menu_data)
    conn.commit()
    print("Menu data inserted successfully.")


except Exception as e:
    print("An error occurred:", e)

finally:
    if conn:
        conn.close()
    if cursor:
        cursor.close()

Drop all table successfully
Table menu created successfully.
Table orders created successfully.
Table order_items created successfully.
Menu data inserted successfully.
